In [1]:
import pandas as pd
import os
import boto3
import numpy as  np
import pandas as pd
import json
import time
from multiprocessing.dummy import Pool

In [2]:
# setup
os.chdir('//FS2\RTO\\CIS-PD MUSC\\')
client = boto3.client('athena', 
                      region_name='us-west-2')
user_id_pairings = pd.read_csv("decoded_forms\\videoID.csv")
subjenroll = pd.read_sas("subjenrollment.sas7bdat")
startstop = pd.read_csv("decoded_forms\\WatchStartStop.csv")
subjenroll.head()

,zSubjectCode,zFoxInsightID,SiteID,ConsentBeforeEnrollment,Gender,Race,Ethnicity,Age,ConsentDt,EnrollmentDt
0,1000.0,b'cisuaba1',1313.0,1.0,1.0,5.0,2.0,63.0,2017-06-15,2017-06-15
1,1001.0,b'cisuabb2',1313.0,1.0,1.0,5.0,2.0,64.0,2017-06-15,2017-06-15
2,1002.0,b'cisuabc3',1313.0,1.0,1.0,5.0,2.0,51.0,2017-06-16,2017-06-16
3,1003.0,b'cisuabd4',1313.0,1.0,2.0,3.0,2.0,60.0,2017-06-19,2017-06-19
4,1004.0,b'cisnwa1',1332.0,1.0,1.0,5.0,2.0,52.0,2017-06-21,2017-06-21


In [3]:
startstop.head()

,SubjCode,Start Year,Start Month,Stop Year,Stop Month
0,142557.0,2017.0,6.0,2017.0,12.0
1,142558.0,2017.0,6.0,2017.0,9.0
2,142559.0,2017.0,6.0,2017.0,7.0
3,142560.0,2017.0,6.0,2017.0,7.0
4,142561.0,2017.0,6.0,2017.0,10.0


In [4]:
paginator = client.get_paginator('get_query_results')

In [5]:
# get the 4 digit user_id from the corresponding 6 digit user_id
def user_id_6_to_4(user_id_6):
    for index, row in user_id_pairings.iterrows():
        if (row['Subj ID Athena']==user_id_6):
            return int(row['SubjectCode'])
    return None

In [6]:
# get the 6 digit user_id from the corresponding 4 digit user_id
def user_id_4_to_6(user_id_4):
    for index, row in user_id_pairings.iterrows():
        if (row['SubjectCode']==user_id_4):
            if np.isnan(row['Subj ID Athena']):
                return None
            return int(row['Subj ID Athena'])
    return None

In [7]:
# generate query from row in form509
def generate_query(YrMoSt,YrMoSp,ID,ID4):
    template1 = "SELECT user_id-"+str(int(ID)-int(ID4))+" AS SubjID, timestamp AS Timestamp, x AS X, y AS Y, z AS Z\n"
    template2 = "FROM watch_accelerometer\nwhere cohort != 'test'\nand user_id = "
    template3 = "\nand timestamp >= timestamp '"
    template4 = "'\nand timestamp < timestamp '"
    template5 = "'"
    
    startTime =  YrMoSt + '-01 00:00:00'
    endTime = YrMoSp + '-01 00:00:00'
    
    query = template1 + template2 + ID + template3 + startTime + template4 + endTime + template5
    return query

In [8]:
# generate query from row in form509
def generate_short_query(YrMoSt,YrMoSp,ID):
    template1 = "SELECT timestamp AS Timestamp, x AS X, y AS Y, z AS Z\nFROM watch_accelerometer\nwhere cohort != 'test'\nand user_id = "
    template2 = "\nand timestamp >= timestamp '"
    template3 = "'\nand timestamp < timestamp '"
    template4 = "'"
    
    startTime =  YrMoSt + '-01 00:00:00'
    endTime = YrMoSt + '-01 08:00:00'
    
    query = template1 + template2 + ID+ startTime + template3 + endTime + template4
    return query

In [9]:
# call query
def call_query(query):
    database='clinician_input_study'
    output_location='s3://aws-athena-query-results-arunjayaraman-us-west-2/'
    encryption_option='SSE_S3'

    query_info = client.start_query_execution(
        QueryString=query, 
        QueryExecutionContext={ 
            'Database': database
        },
        ResultConfiguration={ 
            'OutputLocation': output_location,
            'EncryptionConfiguration': { 
                'EncryptionOption': encryption_option
            }
        }
    )
    return query_info

In [10]:
def removeVarCharValue(dct):
    if 'VarCharValue' in dct:
        return dct['VarCharValue']
    if 'Data' in dct:
        d = {}; val = dct['Data']
        for ind, name in zip(range(4), ['Timestamp', 'X', 'Y', 'Z']):        
            d[name] = val[ind]
        return d 
    return dct

In [11]:
def format_and_save(results,filename,S,First):
    results = list(np.concatenate(results))
    
    # format the results
    if len(results)>1:
        columns_names = [list(d.values())[0] for d in results[0]['Data']]
        Data = pd.read_json(json.dumps(json.loads(json.dumps(results[1::]), object_hook=removeVarCharValue)))
        
        Data.rename(columns={'x':'X','y':'Y','z':'Z','user_id':'SubjID','timestamp':'Timestamp'},inplace=True)
        Data.SubjID = int(S)
        Data = Data.loc[:,['SubjID', 'Timestamp', 'X', 'Y', 'Z']]
        Data.sort_values('Timestamp',inplace=True)
        
        if First:
            Data.to_csv(filename,index = False)
        else:
            Data.to_csv(filename,index = False, header = False, mode = 'a')

In [22]:
# return the results of the query associated with query_info
def get_query_results(query_info, filename):
    query_id = query_info['QueryExecutionId']
    n = 0
    
    query_done=False
    
    while not query_done:
        time.sleep(5)
        query_exec = client.get_query_execution(QueryExecutionId=query_id)
        status = query_exec['QueryExecution']['Status']['State']
        if status=='SUCCEEDED':
            query_done=True
        if (status=='FAILED' or status=='CANCELLED'):
            print('QUERY FAILED')
            return
        
#     First = True
#     results = []
#     nextToken=None

#     while True:
#         if nextToken!=None:
#             page_iter = paginator.paginate(QueryExecutionId=query_id,PaginationConfig={'StartingToken': nextToken})
#         else:
#             page_iter = paginator.paginate(QueryExecutionId=query_id)
#         try:
#             for page in page_iter:
#                 if 'NextToken' in page.keys():
#                     nextToken = page['NextToken']
#                 response = page['ResultSet']['Rows']
#                 results.append(response)
#                 n = n+1
#                 if np.mod(n,100)==0:
#                     results = list(np.concatenate(results))

#                     # format the results
#                     if len(results)>1:
#                         Data = pd.read_json(json.dumps(json.loads(json.dumps(results[1::]), object_hook=removeVarCharValue)))

#                         if First:
#                             Data.to_csv(filename,index = False)
#                             First = False
#                         else:
#                             Data.to_csv(filename,index = False, header = False, mode = 'a')
#                     results = []
#             break
#         except client.exceptions.ClientError:
#             print('Throttling -> Sleep for 5 sec')
#             time.sleep(5)
#             continue
#         break
    
#     # format the leftover results
#     if len(results)>1:
#         results = list(np.concatenate(results))
#         Data = pd.read_json(json.dumps(json.loads(json.dumps(results[1::]), object_hook=removeVarCharValue)))
        
#         Data['SubjID'] = int(S)
#         Data = Data.loc[:,['SubjID', 'Timestamp', 'X', 'Y', 'Z']]
#         Data.sort_values('Timestamp',inplace=True)
        
#         if First:
#             Data.to_csv(filename,index = False)
#         else:
#             Data.to_csv(filename,index = False, header = False, mode = 'a')

In [23]:
# get visit num from the visit information
def get_visit_num(VisitNm):
    if VisitNm.startswith("2 Weeks"):
        if VisitNm.endswith(" 0"):
            return '0'
        elif VisitNm.endswith("30"):
            return '1'
        elif VisitNm.endswith("60"):
            return '2'
        elif VisitNm.endswith("90"):
            return '3'
        elif VisitNm.endswith("120"):
            return '4'
        elif VisitNm.endswith("150"):
            return '5'
    else:
        return '6'

In [24]:
def iterate_subjects(SubjList):
    path = '//FS2.smpp.local\\RTO\\CIS-PD Study\\MJFF Curation\\Finalized Dataset\\'
    for S in SubjList:
        print(S)
        S6 = str(user_id_4_to_6(S))
        if S6!='None':
            inds = startstop.SubjCode==int(S6)
            if sum(inds)<1:
                continue
            row = startstop[inds].iloc[0,:]
            StYr = int(row['Start Year'])
            StMo = int(row['Start Month'])

            Yr = StYr
            Mo = StMo

            SpYr = int(row['Stop Year'])
            SpMo = int(row['Stop Month'])

            Yr_Str = str(Yr)
            Mo_Str = str(Mo) if Mo>=10 else '0'+str(Mo)
            YrMoSt_Str = Yr_Str +'-'+ Mo_Str
            
            while Mo!=SpMo:
                Mo = Mo + 1
                if Mo==13:
                    Mo = 1; Yr = Yr + 1
                Yr_Str = str(Yr)
                Mo_Str = str(Mo) if Mo>=10 else '0'+str(Mo)
                YrMoSp_Str = Yr_Str +'-'+ Mo_Str
                
                query = generate_query(YrMoSt_Str,YrMoSp_Str,S6,S)
                print(path+'Table8\\Table8_'+str(int(S))+'_'+YrMoSt_Str+'.csv')
                if not os.path.exists(path+'Table8\\Table8_'+str(int(S))+'_'+YrMoSt_Str+'.csv'):
                    query_info = call_query(query)
                    get_query_results(query_info,path+'Table8\\Table8_'+str(int(S))+'_'+YrMoSt_Str+'.csv')
                
                YrMoSt_Str = YrMoSp_Str

            Mo = Mo + 1
            if Mo==13:
                Mo = 1; Yr = Yr + 1
            Yr_Str = str(Yr)
            Mo_Str = str(Mo) if Mo>=10 else '0'+str(Mo)
                
            YrMoSp_Str = Yr_Str +'-'+ Mo_Str

            query = generate_query(YrMoSt_Str,YrMoSp_Str,S6,S)
            print(path+'Table8\\Table8_'+str(int(S))+'_'+YrMoSt_Str+'.csv')
            if not os.path.exists(path+'Table8\\Table8_'+str(int(S))+'_'+YrMoSt_Str+'.csv'):
                query_info = call_query(query)
                get_query_results(query_info,path+'Table8\\Table8_'+str(int(S))+'_'+YrMoSt_Str+'.csv')

In [25]:
N = 1
n = 1
SubjList = set(subjenroll.zSubjectCode.unique())
chunkSize = len(SubjList)/N

SubjList = list(SubjList)[(int((n-1)*chunkSize)):int((n*chunkSize))]
print(SubjList)
iterate_subjects(SubjList)

[1024.0, 1025.0, 1026.0, 1027.0, 1028.0, 1029.0, 1030.0, 1031.0, 1032.0, 1033.0, 1034.0, 1035.0, 1036.0, 1037.0, 1038.0, 1039.0, 1040.0, 1041.0, 1042.0, 1043.0, 1044.0, 1045.0, 1046.0, 1047.0, 1048.0, 1049.0, 1050.0, 1051.0, 1052.0, 1053.0, 1054.0, 1055.0, 1056.0, 1000.0, 1001.0, 1002.0, 1003.0, 1004.0, 1005.0, 1006.0, 1007.0, 1008.0, 1009.0, 1010.0, 1011.0, 1013.0, 1014.0, 1015.0, 1016.0, 1017.0, 1018.0, 1019.0, 1020.0, 1021.0, 1022.0, 1023.0]
1024.0
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1024_2017-07.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1024_2017-08.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1024_2017-09.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1024_2017-10.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1024_2017-11.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized D

//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1039_2017-12.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1039_2018-01.csv
1040.0
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1040_2017-07.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1040_2017-08.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1040_2017-09.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1040_2017-10.csv
1041.0
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1041_2017-07.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1041_2017-08.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1041_2017-09.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1041_2017-10.csv
//FS2.smpp.local

//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1009_2017-09.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1009_2017-10.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1009_2017-11.csv
1010.0
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1010_2017-06.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1010_2017-07.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1010_2017-08.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1010_2017-09.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1010_2017-10.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1010_2017-11.csv
//FS2.smpp.local\RTO\CIS-PD Study\MJFF Curation\Finalized Dataset\Table8\Table8_1010_2017-12.csv
1011.0
//FS2.smpp.local

In [15]:
query_id='88a5072b-13b4-453d-8f78-2601bbcca446'
query_exec = client.get_query_execution(QueryExecutionId=query_id)
status = query_exec['QueryExecution']['Status']['State']

In [16]:
status

'SUCCEEDED'

In [19]:
def DownloadQuery(QID):
    get_query_results(QID,'E:\\Table8\\'+QID+'.csv')

In [20]:
DownloadQuery(query_id)

KeyboardInterrupt: 

In [21]:
pd.read_csv('//FS2.smpp.local\\RTO\\CIS-PD Study\\Subjects\\1016\\anterior_thigh_left\\d5la7xew\\2017-07-27T14-24-01-720Z\\gyro.csv')

,Timestamp (ms),Gyro X (°/s),Gyro Y (°/s),Gyro Z (°/s)
0,1501165441720,2.716106,-1.373312,-0.030518
1,1501165441736,3.204395,-1.129168,-0.091554
2,1501165441752,3.448539,-0.823987,-0.152590
3,1501165441768,3.631647,-0.518807,-0.274662
4,1501165441784,3.570611,-0.335698,-0.274662
5,1501165441800,3.509575,-0.457771,-0.213626
6,1501165441816,3.509575,-0.518807,-0.030518
7,1501165441832,3.265431,-0.396735,0.030518
8,1501165441848,3.143359,-0.274662,-0.091554
9,1501165441864,3.143359,-0.274662,-0.152590
